In [1]:
import os
import sys
import torch

from rag_utils import RAG_ut

import transformers
from sentence_transformers import SentenceTransformer

from tqdm import tqdm

import spacy
nlp = spacy.load("de_core_news_lg")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"using {device} as device")

Device 0: NVIDIA RTX A5000
using cuda as device


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Hugging Face Access Token (replace with your own)

# Model ID from Hugging Face Hub
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load tokenizer and model from Hugging Face Hub (requires access token)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    temperature=0.3,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Formulate fictional answer candidates

In [3]:
def reformulate(question):
    sys_prompt = 'Du bist ein Assistent der Anweisungen an ein Suchsystem über Firmendaten in Beispielantworten umwandelt, wie sie auf einer Firmenwebsite stehen könnten. '
    sys_prompt += 'Jede Antwort von Dir besteht aus genau einem Satz. Jeder Satz benutzt dabei typische Schlagwörter um der Suchanfrage zu genügen.'
    sys_prompt += 'Die Frage "Welche Firma beschäftigt sich mit Softwareentwicklung?" könntest Du beantworten mit "Wir sind Experten für Such- und Recherchetechnologien auf natürlichsprachigen Dokumenten."'

    # user_prompt = f'"Question: {question}" | Context: {chunk} \nAnswer:'
    user_prompt = question
    # print(user_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )

    answer = outputs[0]["generated_text"][-1]["content"]    
    if answer.startswith("I don't know"):
        return None
    else:
        return answer

for i in range(5):
    print(reformulate("Ich suche nach Krankenhäusern in der Region."))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Wir bieten eine umfassende Liste von Krankenhäusern in der Region, einschließlich medizinischer Fachabteilungen und Spezialisten für verschiedene Gesundheitsbereiche.
Unsere medizinische Einrichtung bietet umfassende Gesundheitsdienste in der Region an, einschließlich chirurgischer Behandlungen und medizinischer Versorgung.
Wir bieten eine umfassende Liste von Krankenhäusern und medizinischen Einrichtungen in deiner Region an, die sich auf hochwertige Patientenversorgung und medizinische Behandlungen spezialisieren.
Wir bieten eine umfassende Liste von Spitälern und medizinischen Einrichtungen in deiner Region, einschließlich Kliniken, Fachkliniken und Gesundheitszentren.
Unser Netzwerk umfasst führende Gesundheitsinstitute und Krankenhäuser in der Region, die sich auf hochqualifizierte medizinische Dienstleistungen spezialisiert haben.


In [4]:
def reformulate(question, reformulate_sents_n=5):
    sys_prompt = 'Du bist ein Assistent der Anweisungen an ein Suchsystem über Firmendaten in Beispielantworten umwandelt, wie sie auf einer Firmenwebsite stehen könnten. '
    sys_prompt += f'Jede Antwort von Dir besteht aus genau {reformulate_sents_n} Sätzen. Jeder Satz für sich genommen, könnte so auf einer Firmenwebsite zu finden sein. '
    sys_prompt += 'Jeder Satz benutzt dabei typische Schlagworte oder Buzz-Words um der Suchanfrage zu genügen. Die Sätze sollten unterschiedlich sein. '
    sys_prompt += 'Du vermeidest allgemeine generische Formulierungen, die nicht spezifisch für für die Anfrage sind, wie z.B.: "Unsere Kunden schätzen unsere schnelle Reaktionszeit und unsere professionelle Kommunikation.".  '
    sys_prompt += 'Die Frage "Welche Firma beschäftigt sich mit Softwareentwicklung?" könntest Du beantworten mit "Wir sind Experten für Such- und Recherchetechnologien auf natürlichsprachigen Dokumenten."'

    # user_prompt = f'"Question: {question}" | Context: {chunk} \nAnswer:'
    user_prompt = question
    # print(user_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )

    answer = outputs[0]["generated_text"][-1]["content"]    
    doc = nlp(answer)
    return [s.text.strip() for s in doc.sents]
    

print("\n".join(reformulate("Ich suche nach Krankenhäusern in der Region.")))

Unser Krankenhaus bietet eine breite Palette medizinischer Dienstleistungen an, um die Gesundheit unserer Patienten zu fördern.
Wir haben eine erfahrenen Ärzteteam, das sich auf die neuesten medizinischen Entwicklungen und Behandlungsmethoden spezialisiert hat.
Unsere modernen Krankenhausgebäude sind mit den neuesten medizinischen Technologien ausgestattet, um eine sichere und effektive Patientenversorgung zu gewährleisten.
Wir bieten eine Vielzahl von medizinischen Fachabteilungen an, darunter Chirurgie, Gynäkologie, Onkologie und mehr.
Durch unsere enge Zusammenarbeit mit anderen Gesundheitsdienstleistern können wir eine umfassende medizinische Versorgung anbieten.


## Vector Search using answer candidates

In [5]:
from collections import defaultdict


# model = SentenceTransformer("all-MiniLM-L6-v2")
model = SentenceTransformer("PM-AI/bi-encoder_msmarco_bert-base_german")

qdrant_collection_name = "ies_website_embeddings_msmarco_german" 
# qdrant = Qdrant("172.18.0.4", 6333, "ies_website_embeddings", model)
qdrant = Qdrant("http://qdrant:6333", qdrant_collection_name, model)

t = qdrant.query("Ich benötige eine Software für die Unterstützung des regulatorischen Managements.", 1)
print(t[0], "\n")
t[0].payload

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


id='f139d2f8-d71f-499f-9f33-c3bbd13e1037' version=171679 score=0.9745883 payload={'filepath': '/local/innecs-local/website-data/3170118882_regiocom-com/regiocom-energie-regulierungsmanagement.txt', 'filename': 'regiocom-energie-regulierungsmanagement.txt', 'crefoid': '3170118882', 'name': 'regiocom SE', 'idx': 2, 'text': 'Die Software unterstützt bei allen Fragestellungen im Bereich des Regulierungsmanagements.', 'crefodata': {'file': 'cache/39112/82/3170118882_REGIOCOM_SE.html', 'filetime': '2023-08-29T09:47:28.320Z', 'origin': 'https://firmeneintrag.creditreform.de/39112/3170118882/REGIOCOM_SE', 'crefonummer': '3170118882', 'postalcode': '39112', 'code': '96.09.0', 'zweck': 'Entwicklung und Vermarktung von Dienstleistungen auf der Basis moderner Informations- und Kommunikationstechnik und deren Betrieb.', 'name': 'regiocom SE', 'desc': 'regiocom SE mit Sitz in Magdeburg ist im Handelsregister mit der Rechtsform Societas Europaea eingetragen. Das Unternehmen wird beim Amtsgericht 3957

{'filepath': '/local/innecs-local/website-data/3170118882_regiocom-com/regiocom-energie-regulierungsmanagement.txt',
 'filename': 'regiocom-energie-regulierungsmanagement.txt',
 'crefoid': '3170118882',
 'name': 'regiocom SE',
 'idx': 2,
 'text': 'Die Software unterstützt bei allen Fragestellungen im Bereich des Regulierungsmanagements.',
 'crefodata': {'file': 'cache/39112/82/3170118882_REGIOCOM_SE.html',
  'filetime': '2023-08-29T09:47:28.320Z',
  'origin': 'https://firmeneintrag.creditreform.de/39112/3170118882/REGIOCOM_SE',
  'crefonummer': '3170118882',
  'postalcode': '39112',
  'code': '96.09.0',
  'zweck': 'Entwicklung und Vermarktung von Dienstleistungen auf der Basis moderner Informations- und Kommunikationstechnik und deren Betrieb.',
  'name': 'regiocom SE',
  'desc': 'regiocom SE mit Sitz in Magdeburg ist im Handelsregister mit der Rechtsform Societas Europaea eingetragen. Das Unternehmen wird beim Amtsgericht 39576 Hansestadt Stendal unter der Handelsregister-Nummer HRB 2

In [6]:
def max_scores(grouped_data):
    max_scores = {}
    for compkey in grouped_data.keys():
        max_score = 0
        name = ""
        filedata = grouped_data[compkey]
        for filekey in filedata.keys():
            for data in filedata[filekey]:
                max_score = max(max_score, data["score"])
                name = data["name"]

        max_scores[name] = max_score

    return max_scores
            

def find_similar(text, reformulate_sents_n=5, qdrant_min_sim = 0):
    formulations = reformulate(text, reformulate_sents_n)
    formulations.append(text)

    print("\n".join(formulations), "\n\n")

    all_hits = []
    
    for f in formulations:        
        hits = qdrant.query(f)
        
        for hit in hits:
            if hit.score <= qdrant_min_sim:
                continue
            crefoid = hit.payload["crefoid"]
            filepath = hit.payload["filepath"]
            idx = hit.payload["idx"]
            name = hit.payload["name"]
            text = hit.payload["text"]
            web = hit.payload["crefodata"]["web"]

            all_hits.append({
                "crefoid": crefoid,
                "name": name,
                "filepath": filepath,
                "idx": idx,
                "text": text,
                "web": web,
                "score": hit.score,                
            })
            
    grouped_data = defaultdict(lambda: defaultdict(list))
    for result in all_hits:
        grouped_data[result['crefoid']][result['filepath']].append({
            'idx': result['idx'],
            'text': result['text'],
            'name': result['name'],
            'score': result['score'],
            'web': result['web']
        })

    grouped_data = {k: dict(v) for k, v in grouped_data.items()}    

    return grouped_data

# results = find_similar("Ich benötige eine Software für die Unterstützung des regulatorischen Managements.", 0.75)
query = "Ich benötige eine Software für die Unterstützung des regulatorischen Managements."
results = find_similar(query, 0.75)
print("max_scores:")
print(max_scores(results),"\n\n")
results


Wir bieten ein umfassendes Compliance-Management-System, das Unternehmen bei der Erfüllung regulatorischer Anforderungen unterstützt.
Mit unserem Tool können Sie Risiken identifizieren, Prozesse optimieren und Compliance-Verpflichtungen effizient überwachen.
Ich benötige eine Software für die Unterstützung des regulatorischen Managements. 


max_scores:
{'Elektro-Thermit GmbH &amp; Co. KG': 0.9887617, 'Elektro-Thermit Verwaltungs-GmbH': 0.9887617, 'ALDI GmbH &amp; Co. Kommanditgesellschaft': 0.9881476, 'Altmark-Käserei Uelzena Gesellschaft mit beschränkter Haftung': 0.9854971, 'Thermowave Gesellschaft für Wärmetechnik mbH': 0.98521256, 'Robin Data GmbH': 0.9849719, 'Semcoglas GmbH': 0.98430634, 'TRG Cyclamin GmbH': 0.9842732, 'CONDOR Sicherungs- und Service GmbH &amp; Co. KG, Nord-Ost': 0.984208, 'valantic Transaction Solutions GmbH': 0.9839498, 'VERBIO Pinnow GmbH': 0.983623, 'VBB Verwaltungs- und Beteiligungsgesellschaft mbH': 0.98338866, 'Volks- und Raiffeisenbank Saale-Unstrut eG':

{'3130006858': {'/local/innecs-local/website-data/3130006858_elektro-thermit-de/compliance.txt': [{'idx': 8,
    'text': 'Unser Compliance-Management-System hilft uns sicherzustellen, dass unsere Geschäftsaktivitäten stets ethischen, gesetzlichen und professionellen Standards entsprechen.',
    'name': 'Elektro-Thermit GmbH &amp; Co. KG',
    'score': 0.9887617,
    'web': 'www.elektro-thermit.de'}]},
 '3130253636': {'/local/innecs-local/website-data/3130253636_elektro-thermit-de/compliance.txt': [{'idx': 8,
    'text': 'Unser Compliance-Management-System hilft uns sicherzustellen, dass unsere Geschäftsaktivitäten stets ethischen, gesetzlichen und professionellen Standards entsprechen.',
    'name': 'Elektro-Thermit Verwaltungs-GmbH',
    'score': 0.9887617,
    'web': 'www.elektro-thermit.de'}]},
 '3170089382': {'/local/innecs-local/website-data/3170089382_aldi-nord-de/unternehmen-ueber-aldi-nord-html.txt': [{'idx': 24,
    'text': 'Unser Compliance-Management-System hilft uns, unsere

## Augment Context for Generation Process

for each company:
- use all files in answer candidates
- add *pre_n* sentences before the answer (if possible)
- add *post_n* sentences after the answer (if possible)

In [7]:
text = qdrant.get_sentence_range("3170353356", "/local/innecs-local/website-data/3170353356_legalhorizon-de/_.txt", 2, 7, "\n---\n")
print(text)

Zum Inhalt scrollen
HOME
---
Warum wir Sie ruhiger schlafen lassen
---
Die Herausforderung der regulatorischen Compliance

Die Pflicht zum regulatorischen Monitoring trifft die Geschäftsleitung nicht nur aus dem Gebot zur Legalitätskontrolle (für Deutschland z.B. §§ 76 I, 93 I AktG, § 43 I GmbHG).
---
Auch spezialgesetzlich, z.B. aus den Mindestanforderungen an das Risikomanagement der Banken (MaRisk), dem Versicherungsaufsichtsgesetz oder der Gewerbeordnung folgt die Notwendigkeit sich kontinuierlich über neue und künftige Regularien zu informieren, die entsprechenden Schlussfolgerungen zu ziehen, abgeleitete Maßnahmen umzusetzen und all dies zu dokumentieren.
---
In dritten Märkten


In [8]:
print(qdrant.augment_context("3170353356", "/local/innecs-local/website-data/3170353356_legalhorizon-de/_.txt", 10, 1, 3, "\n---\n"))

Vorgaben werden komplexer, Vorschriften kleinteiliger.
---
Auch mangelnde Rechtskonformität hat zunehmend drastischere Folgen.
---
Neben einer persönlichen Haftung für die Geschäftsleitung werden Rechtsverstöße zunehmend mit Sanktionen belegt, die sich auch an den unternehmensweiten Umsätzen orientieren.
---
​
---
Mit der Digitalisierung verbunden ist eine zunehmende Beschleunigung der Geschäftsprozesse.


In [9]:
def answer(query, company_name, context):
    sys_prompt = 'Du bist ein Assistent der Nutzeranfragen anhand des Contexts einer Unternehmenswebsite überprüft. '
    sys_prompt += 'Alle Anfragen werden Dir in dem Format "Frage: <Frage> | Name: <Name des Unternehmens> | Context: <Context einer Unternehmenswebsite>" übergeben. '
    sys_prompt += 'Du bewertest anhand des Contexts inwiefern das Unternehmens für die Anfrage des Nutzers relevant ist. '
    sys_prompt += 'Du vergibst dafür 0 bis 9 Sterne, wobei 9 "sehr relevant" bedeutet und 0 "gar nicht relevant bedeutet". '    
    sys_prompt += 'Deine Antwort leitest Du immer mit der Sternanzahl ein und gibst dann eine Zusammenfassung, z.B.: "3: <Zusammenfassung>". '
    sys_prompt += 'Die Zusammenfassung soll kurz und knapp ausfallen, aber die wichtigsten Fakten bzgl der Nutzeranfrage enthalten. '
    

    user_prompt = f'Frage: {query} | Name: {company_name} | Context: {context}'    

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )

    answer = outputs[0]["generated_text"][-1]["content"]    
    return answer

def rag_query(query, augment_sent_n=5, qdrant_min_score=0.75, ):
    query = query
    similar_sents = find_similar(query, augment_sent_n, qdrant_min_score)

    scored_answers = []
    for crefoid in tqdm(similar_sents.keys()):
        company_data = similar_sents[crefoid]
        company_name = ""
        web=""
        
        context = set()
        for file in company_data.keys():
            for entry in company_data[file]:
                company_name = entry["name"]
                web = entry["web"]
                context.add(qdrant.augment_context(crefoid, file, entry["idx"], 5, 10))
                
        
        a = answer(query, company_name, " ".join(list(context))).strip()        
        scored_answers.append((a[0], company_name, web, a[2:].strip()))

    return sorted(scored_answers, key=lambda x: x[0], reverse=True)

rag_answers = rag_query("Die IT-Abteilung unseres Unternehmens wurde mit Ransomware gehackt. Wer kann mir helfen?")
rag_answers

Unser Team von IT-Sicherheitsexperten ist spezialisiert auf die Bekämpfung von Cyberbedrohungen und Ransomware-Angriffen.
Wir bieten umfassende Lösungen für die Wiederherstellung von Daten und Systemen.
Unsere Experten sind zertifiziert in der Verwendung von Sicherheitssoftware und -technologien.
Wir arbeiten eng mit unseren Kunden zusammen, um ihre IT-Sicherheit zu stärken und potenzielle Schwachstellen zu identifizieren.
Durch unsere schnelle Reaktion und professionelle Kommunikation können wir unsere Kunden schnell wieder auf die Beine helfen.
Die IT-Abteilung unseres Unternehmens wurde mit Ransomware gehackt. Wer kann mir helfen? 




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [06:21<00:00,  4.95s/it]


[('9',
  'deflex IT GmbH',
  'www.deflex-it.de',
  'Die deflex IT GmbH bietet spezielle Dienstleistungen im Bereich der Datenrettung und IT-Wartung an, einschließlich der Rettung von Daten von defekten Festplatten, Reparatur von defekten IT-Geräten und schneller Hilfe bei IT-Problemen. Da die Anfrage des Nutzers sich direkt auf die Dienstleistungen der deflex IT GmbH bezieht, ist das Unternehmen sehr relevant für die Anfrage.'),
 ('9',
  'comstab SYSTEMHAUS GmbH',
  'www.comstab.de',
  'Die comstab SYSTEMHAUS GmbH ist sehr relevant für die Anfrage des Nutzers, da sie sich auf IT-Services spezialisiert hat und auch Ransomware-Angriffe und Virenentfernung anbietet. Die Website des Unternehmens erwähnt explizit, dass sie sich um die Datenrettung und Virenentfernung kümmert, falls ein Wurm, Virus oder Trojaner eingeschlichen ist. Außerdem betonen sie die Bedeutung der Sicherheit der IT-Systeme und bieten professionelle Lösungen an, um Daten vor Verlust zu schützen.'),
 ('9',
  'QUANTOR CON

In [10]:
rag_answers = rag_query("Ich benötige eine Software für die Unterstützung des regulatorischen Managements.")
rag_answers

Unser Unternehmen bietet eine umfassende Lösung für das regulatorische Management, die Unternehmen dabei unterstützt, Compliance-Vorschriften einzuhalten und Risiken zu minimieren.
Mit unserer Software können Unternehmen komplexe Vorschriften und Richtlinien effizient verwalten und sicherstellen, dass alle Anforderungen erfüllt werden.
Unsere Lösung ermöglicht es Unternehmen, Prozesse zu automatisieren, Dokumente zu verwalten und Risiken zu identifizieren.
Durch die Verwendung unserer Software können Unternehmen ihre Compliance-Strategie effektiv umsetzen und sich auf strategische Ziele konzentrieren.
Durch unsere Lösung können Unternehmen ihre Compliance-Strategie effektiv umsetzen und sich auf strategische Ziele konzentrieren.
Ich benötige eine Software für die Unterstützung des regulatorischen Managements. 




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [05:10<00:00,  5.45s/it]


[('9',
  'Soft-Trade GmbH',
  'www.soft-trade.de',
  'Die Soft-Trade GmbH bietet eine Software für das regulatorische Management, insbesondere für Versicherungs- und Finanzmakler, an. Die Firma hat langjährige Partnerschaften mit starken Produktpartnern, wie der ELO Digital Business GmbH, und bietet innovative Lösungen wie Keasy und ELO Realität an, die die Arbeitsprozesse automatisieren und Daten sicher speichern. Die Firma unterstützt ihre Kunden bei der Organisation ihrer täglichen Arbeit und der Verwaltung ihrer Daten und Dokumente. Die Anfrage des Nutzers, eine Software für das regulatorische Management, ist daher sehr relevant für die Soft-Trade GmbH.'),
 ('9',
  'LegalHorizon AG',
  'www.legalhorizon.de',
  'Die LegalHorizon AG ist sehr relevant für die Anfrage des Nutzers, da sie sich speziell auf die Unterstützung des regulatorischen Managements durch ihre Softwarelösung konzentriert. Die Anwendung der LegalHorizon AG-Software ermöglicht Unternehmen die Überwachung und Analyse

In [11]:
rag_answers = rag_query("Wer kann medizinische Atemschutzmasken herstellen?")
rag_answers

Wir sind ein führender Hersteller von medizinischen Atemschutzmasken, die auf höchstem Qualitätsstandard hergestellt werden.
Unsere Masken sind für den Einsatz in verschiedenen medizinischen Bereichen geeignet, von der Intensivpflege bis hin zur Notfallmedizin.
Wir setzen auf fortschrittliche Materialien und innovative Designs, um unsere Produkte sicher und effektiv zu machen.
Unsere Produktlinie umfasst eine Vielzahl von Atemschutzmasken, von einfachen Gesichtsmasken bis hin zu komplexen Beatmungsgeräten.
Durch unsere enge Zusammenarbeit mit medizinischen Experten stellen wir sicher, dass unsere Produkte den höchsten Anforderungen entsprechen.
Wer kann medizinische Atemschutzmasken herstellen? 




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [05:04<00:00,  5.34s/it]


[('9',
  'GENUVIS GmbH',
  'www.genuvis.de',
  'Die GENUVIS GmbH ist sehr relevant für die Anfrage des Nutzers, da sie speziell medizinische Atemschutzmasken herstellt und anbietet, einschließlich KN95-Masken, die eine Filterleistung von über 95% haben. Die Firma bietet verschiedene Arten von Masken an, darunter 3-lagige Einwegmasken, Mund-Nasen-Masken KN95 und OP-Masken, und liefert sie an Ärzte, Kliniken und Privatpersonen. Die Masken sind lagernd in Deutschland und werden sofort versandt.'),
 ('8',
  'Hendrik Schirmer &amp; Thomas Zitzl GbR  Outdoorgeschäft  fluchtrucksack.de',
  'www.fluchtrucksack.de',
  'Die Website von Fluchtrucksack.de bietet eine breite Palette an Produkten für Krisenvorsorge und Selbstschutz, einschließlich Atemschutzmasken. Der Anbieter Ekastu ist ein renommierter Hersteller von Atemschutzmasken und Schutzbekleidung, der höchste Qualität und Sicherheit gewährleistet. Die Website bietet eine große Auswahl an Produkten, die speziell für den Einsatz in Notsitua

In [12]:
rag_answers = rag_query("Ich suche nach japanischen Restaurants.")
rag_answers

Unser Restaurant bietet authentische japanische Küche mit einer Vielzahl von traditionellen Gerichten.
Wir verwenden nur frische Zutaten und importieren sie direkt aus Japan, um unsere Gerichte authentisch zu halten.
Unsere Spezialitäten umfassen Sushi, Sashimi und Ramen, die von unseren erfahrenen Köchen zubereitet werden.
Wir bieten auch eine umfangreiche Auswahl an japanischen Getränken an, einschließlich Sake und Mugi-cha.
In unserem Restaurant können Sie eine einzigartige japanische Essens-Erfahrung genießen.
Ich suche nach japanischen Restaurants. 




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [04:08<00:00,  4.00s/it]


[('9',
  'Travel Charme Wernigerode GmbH',
  'www.travelcharme.com/hotels/gothisches-haus.html',
  'Die Travel Charme Wernigerode GmbH ist sehr relevant für die Anfrage des Nutzers, da auf der Website explizit japanische Restaurants und Sushi-Karten angeboten werden. Der Nutzer kann hier direkt reservieren und das Alpen-Sushi genießen.'),
 ('9',
  'GK Beteiligungsgesellschaft mbH',
  'www.sushifreunde.de',
  'Die Anfrage des Nutzers ist sehr relevant für die Unternehmenswebsite von GK Beteiligungsgesellschaft mbH, da die Website explizit japanische Restaurants und Sushi anbietet. Die Website enthält eine detaillierte Speisekarte, Informationen über ein Mittagsmenü, ein All-you-can-eat Sushi Buffet und einen Sushi Kochkurs. Der Nutzer kann also direkt auf der Website nach Informationen zu japanischen Restaurants suchen und findet alle relevanten Informationen.'),
 ('9',
  'SF MD GmbH',
  'www.sushifreunde.de',
  'Die SF MD GmbH ist sehr relevant für die Anfrage des Nutzers, da die Websi

In [13]:
rag_answers = rag_query("Für die Ausrichtung einer Hochzeit suche ich Event-Agenturen, die damit Erfahrung haben.")
rag_answers

Wir bieten maßgeschneiderte Eventlösungen für Hochzeitsfeiern, die von der Planung bis zur Durchführung perfekt abgestimmt sind.
Unser Team von Event-Profis hat jahrelange Erfahrung in der Organisation von Traumhochzeiten und kennt sich mit allen Details aus.
Wir sind Experten für die Gestaltung von einzigartigen und unvergesslichen Hochzeitsfeiern, die Ihren Wünschen und Vorstellungen entsprechen.
Unsere Event-Agentur bietet eine breite Palette von Dienstleistungen an, von der Dekoration und Musik bis hin zur Logistik und Organisation.
Mit unserer Erfahrung und unseren Kontakten können wir Ihnen eine Hochzeit planen, die Ihre Träume wahr macht und Ihre Gäste begeistert.
Für die Ausrichtung einer Hochzeit suche ich Event-Agenturen, die damit Erfahrung haben. 




100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [04:53<00:00,  5.53s/it]


[('9',
  'Stefanie Krahmer, Dennis Krahmer, Thomas Krahmer, 11 Grad Ost, Eventagentur',
  'www.elfgradost.de',
  'Die Event-Agentur 11 Grad Ost ist sehr relevant für die Anfrage des Nutzers. Die Agentur bietet spezielle Hochzeitsplanung und -organisation an, einschließlich der Auswahl von Location, Dekoration, Catering und Musik. Sie haben auch Erfahrung mit der Planung von Hochzeiten in der VIEHBÖRSE Magdeburg, die im Kontext der Anfrage erwähnt wird. Die Agentur bietet auch individuelle Konzepte und Beratung an, was auf die Anfrage des Nutzers, nach einer Event-Agentur mit Erfahrung in der Hochzeitsplanung, zugeschnitten ist.'),
 ('9',
  'Nadja Vohse Eventschloss Schönfeld',
  'eventschloss-schoenfeld.de',
  'Das Unternehmen Nadja Vohse Eventschloss Schönfeld ist sehr relevant für die Anfrage des Nutzers, da es sich auf die Planung und Durchführung von Hochzeiten spezialisiert hat und eine Vielzahl von Dienstleistungen anbietet, einschließlich der Auswahl der Dekoration, der Koordina